# Uber Prize Starter Kit Python Utilities: Tutorial

To simplify interaction with the Docker-based simulation execution and evaluation, we've provided a set of Python utilities (located in the `/utilities` folder of the Starter-Kit repository). This notebook demonstrates how they may be used to accomplish the following tasks:

 - Starting a simulation or several simulations
 - Checking simulation completion
 - Retrieving the simulation score in a convenient Pandas `DataFrame` format.
 - Generating fake data 

*Note*: It is assumed that this notebook is started from the `/examples` folder.

In [3]:
# Adding the module to the path for future import
import sys
import os
import docker
from pathlib import Path
# Note that the following is idempotent when this notebook is run from "/examples"
os.chdir('../utilities')
%load_ext autoreload
%autoreload 2

## Running a simulation: the `competition_executor` module

A `CompetitionContainerExecutor` object may be used to start, stop, and gather information about containers running simulations and/or completed simulation scores and stats.

In [63]:
from competition_executor import CompetitionContainerExecutor

# Current round uses "sioux faux" scenario
SCENARIO_NAME = "sioux_faux"

# Evaluation is on 15k sample (1k sample also available)
SAMPLE_SIZE = "15k"

path_input = (Path(Path.cwd()).parent / "submission-inputs").absolute()
path_output = (Path(Path.cwd()).parent / "output").absolute()

my_executor = CompetitionContainerExecutor(input_root=path_input,
                                  output_root=path_output)

# Note: Instantiating a CompetitionContainerExecutor with path arguments is not strictly necessary. Each simulation
# can be run with its own set of input and/or output arguments. However, if you prefer to designate
# a single directory for inputs and/or a single directory for outputs, then, for convenience, 
# you may pass those arguments in here and avoid re-entering them for each simulation run.

Run the simulation using the `run_sumulation` method. For example:

In [66]:
# Note: the `submission name` for each run must be unique. However, this cell is designed to be run repeatedly. 
# If a simulation is currently in progress, it will be stopped and removed.
try:
    my_submission = my_executor.client.containers.get('my_submission')
    print("WARNING: The executor found a simulation named '{0}' (container ID '{1}') from a previous run.\n\
    - Stopping simulation and removing the old container.\n \
    - Creating new container and restarting simulator.\n \
          ~~~Please wait a moment~~~".format(my_submission.name, my_submission.short_id))
    my_submission.stop()
    my_submission.remove()
except docker.errors.NotFound:
    print("Creating new container and starting simulator.\n \
    ~~~Please wait a moment~~~\n")
    
my_executor.run_simulation('my_submission', num_iterations=1, num_cpus=72)

# Note that a dictionary indicating which simulation names already exist can be accessed.
for key, value in my_executor.containers.items():
    print("\nSuccessfully created new container and currently executing a simulation run\n \
    - Name: '{0}'\n \
    - Container ID : '{1}'\n \
    \n \
    ~~~Scores will be available when this run completes.~~~\n".format(value.name, value.id))

    - Stopping simulation and removing the old container.
     - Creating new container and restarting simulator.
           ~~~Please wait a moment~~~

Successfully created new container and currently executing a simulation run
     - Name: 'my_submission'
     - Container ID : 'a9b0c3e9f9'
     
     ~~~Scores will be available when this run completes.~~~


Successfully created new container and currently executing a simulation run
     - Name: 'bistro-competition'
     - Container ID : 'b267cfb592fdc1190c63859e02022d334beced510804e765e87c29bd558e65fc'
     
     ~~~Scores will be available when this run completes.~~~


Successfully created new container and currently executing a simulation run
     - Name: 'ecstatic_clarke'
     - Container ID : 'e299dd54475050a790f11c3650e32b6f0b654ff0c6320bf034b356fec7effc88'
     
     ~~~Scores will be available when this run completes.~~~


Successfully created new container and currently executing a simulation run
     - Name: 'condescending_h

You may view (or save to file) the **debug logs** of your simulation using the `output_simulation_logs(simulation_name,file_name)` method on the executor. 

To follow simulation progress, the following cell may be run repeatedly. 

In [63]:
logs = my_executor.output_simulation_logs('my_submission',filename='error_sim.log')

# NOTE: While it may take a while to finish, as long as your execution environment meets 
# the minimum hardware requirements the  simulation is (most likely) not stuck! 

# However, if you notice that the logs do not change for 
# over 5 minutes, there is likely an error somewhere and you should stop the simulation (see below). 
# In this case, we would greatly appreciate it if you filed a bug report (issue) 
# together with your hardware environment (number of cpus, RAM, etc.) and a copy of the output 
# logs (zipped, preferably).

Submissions run for a certain amount of time. If you interrupt it before the end, you will not get any outputs. You can **check if a submission is finished** with the following method:

In [65]:
my_executor.check_if_submission_complete('my_submission')

True

When a simulation run is done, you can import its **results**:

In [66]:
scores, stats = my_executor.get_submission_scores_and_stats('my_submission')

In [1]:
scores['Raw Score']

NameError: name 'scores' is not defined

The **scores** and **statistics** are stored in pandas DataFrames which contains the information described [here](https://github.com/vgolfier/Uber-Prize-Starter-Kit/blob/master/docs/Understanding_the_outputs_and_the%20scoring_function.md).

At the end of any run and to avoid any conflicts between submission ids (particularly if you want to reuse names), it is advised to **clean up containers** calling the following method:

In [ ]:
my_executor.stop_all_simulations()

## The `input_sampler` module:

Randomly sampled data may be used to either initialize the re-planning algorithm or otherwise test the simulation. The `input_sampler` has been provided towards this end. This subsection provides an example of how synthetic random input files may be generated for each of the available input policies to the simulation.

In [59]:
from input_sampler import *
# Specify the common string for the scenario name 
# (currently only "siouxfalls", which refers to the Sioux Falls scenario).
agency = 'sioux_faux_bus_lines'

# Set the paths appropriately
DIR = Path(__name__).absolute().parent.parent
sys.path.append(str(DIR))
DATA_DIR = DIR / 'reference-data/'

agency_dict = scenario_agencies(DATA_DIR,SCENARIO_NAME)
    # Create a lazy cache of GTFS data for the agency:
sf_gtfs_manager = AgencyGtfsDataManager(agency_dict[agency])

# Sample each input five (5) times (naming of functions is indicative of type of input sampled).
freq_df = sample_frequency_adjustment_input(5, sf_gtfs_manager)
mode_incentive_df = sample_mode_incentives_input(5)
vehicle_fleet_mix_df = sample_vehicle_fleet_mix_input(5, sf_gtfs_manager)

# Generated inputs may now be saved in, say, /submission-inputs or some other directory. 
# Remember the location of this directory when executing new simulations.

In [61]:
vehicle_fleet_mix_df

,agencyId,routeId,vehicleTypeId
0,217,1351,BUS-STD-HD
1,217,1340,BUS-DEFAULT
2,217,1342,BUS-SMALL-HD
3,217,1344,BUS-DEFAULT
4,217,1346,BUS-SMALL-HD


In [62]:
mode_incentive_df

,mode,age,income,amount
0,drive_transit,[15:70),[207000:260000),8.3
1,walk_transit,[28:62],[138000:259000],9.1
2,drive_transit,(7:23),[58000:60000],8.9
3,drive_transit,[7:89),[197000:239000),2.3
4,ondemand_ride,(18:69),[7000:174000),19.0


In [63]:
freq_df

,trip_id,start_time,end_time,headway_secs,exact_times
0,t_75367_b_219_tn_3,43260,66000,6000,0
1,t_75351_b_219_tn_3,9780,21000,3420,0
2,t_72465_b_219_tn_2,7080,38700,6720,0
3,t_75323_b_219_tn_1,47580,79200,7080,0
4,t_75356_b_219_tn_1,43740,81120,2760,0
